<a href="https://colab.research.google.com/github/ndvp39/CloudComputing-tirgul/blob/main/searchEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive, output
import IPython
from IPython.display import HTML, Javascript
import json
from IPython.display import clear_output

drive.mount('/content/drive')

In [ ]:


def receive_search_string(search_string):

  clear_output(wait=True)
  css_filename = "/content/drive/My Drive/CloudComputing/project/style.css"
  html_filename = "/content/drive/My Drive/CloudComputing/project/results.html"
  js_filename = "/content/drive/My Drive/CloudComputing/project/script.js"

  with open(css_filename, 'r') as file:
    css_content = file.read();
    display(IPython.display.HTML(css_content))
  with open(html_filename, 'r') as file:
    html_content = file.read();
    display(IPython.display.HTML(html_content))

    js_code = f"""
      <script>
        document.getElementById("searchbartext").value = "{search_string}";
      </script>
      """
  display(HTML(js_code))

  with open(js_filename, 'r') as file:
    js_content = file.read();
    display(IPython.display.Javascript(js_content))



# Register the function to be called from JavaScript
output.register_callback('notebook.receive_search_string', receive_search_string)


In [ ]:
css_filename = "/content/drive/My Drive/CloudComputing/project/style.css"
html_filename = "/content/drive/My Drive/CloudComputing/project/index.html"
js_filename = "/content/drive/My Drive/CloudComputing/project/script.js"

with open(css_filename, 'r') as file:
  css_content = file.read();
  display(IPython.display.HTML(css_content))

with open(html_filename, 'r') as file:
  html_content = file.read();
  display(IPython.display.HTML(html_content))

with open(js_filename, 'r') as file:
  js_content = file.read();
  display(IPython.display.Javascript(js_content))






In [ ]:
html_filename = "/content/drive/My Drive/CloudComputing/project/admin.html"

with open(html_filename, 'r') as file:
  html_content = file.read();
  display(IPython.display.HTML(html_content))